# Прогнозирование стоимости автомобиля по его характеристикам: 
# Часть 2: обучаем модели, сравниваем результаты

*Над проектом работали: Бочкарева Ксения (bochkareva.2014@mail.ru), Журавлев Алексей (alexeizhuravlev@icloud.com)*

In [1]:
from sklearn.ensemble import BaggingRegressor, StackingRegressor
import warnings
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from datetime import timedelta, datetime, date
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from prettytable import PrettyTable
from pandas import Series
from sklearn.feature_selection import f_regression, mutual_info_regression
import xgboost as xgb
from prettytable import PrettyTable

import seaborn as sns
%matplotlib inline


warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

Теперь, когда данные готовы для ML, можно приступать к экспериментам. Начнем с базовой модели - линейной регрессии с дефолтными параметрами. Затем будем пробовать более сложные модели и посмотрим, какая покажет себя лучше всего. В конце поработаем над подбором гиперпараметров.

In [2]:
# полностью обработанные и готовые к работе данные
data = pd.read_csv('final_data_module5.csv')
data = data.drop('Unnamed: 0', axis=1)

sample_submission = pd.read_csv(
    'sample_submission_module5.csv')  # шаблон сабмита на Kaggle

In [3]:
# введем функцию для подсчета ошибки MAPE

def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res


def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

## 1. Базовая модель дефолтной линейной регрессии

In [4]:
# разделим снова на train и test, т.к. все признаки уже обработаны
train_data = data[data['sample'] == 1].drop(['sample'], axis=1)
test_data = data[data['sample'] == 0].drop(['sample'], axis=1)

# выделим целевую переменную и признаки
X = train_data.drop(['Лог_Цена', 'Цена'], axis=1)
y = train_data['Цена']

# разделим на тренировочные и валидационные выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

# прологарифмируем целевую переменную, тренировать модель будем на ней
y_train_log = np.log(y_train)

In [6]:
# обучаем baseline модель
baseline = LinearRegression().fit(X_train, y_train_log)

# делаем "логарифмированое" предсказание
y_pred_log = baseline.predict(X_test)

# приводим цену к нужному формату: экспоненциируем и делаем кратность = 1000
y_pred = (np.exp(y_pred_log)//1000)*1000

# считаем ощибку
# 16.8% ошибка, довольно неплохо для базовой модели
mean_absolute_percentage_error(y_test, y_pred)

16.820687487587392

Пробуем сделать сабмит на платформу:

In [7]:
# пробуем Submission на Kaggle

# берем тестовые данные (признаки)
# данные для тестирование с Kaggle
test_Kaggle = test_data.drop(['Лог_Цена', 'Цена'], axis=1)

# предсказываем на них
y_pred_test_l = baseline.predict(test_Kaggle)
y_pred_test = (np.exp(y_pred_test_l)//1000)*1000

# берем предсказанные цены
sample_submission['price'] = y_pred_test

# записываем в файл
#sample_submission.to_csv('baseline_model.csv', index=False)

# Ошибка 51.5% на Kaggle - плоховато

#### Ошибка на тестовых данных (на платформе Kaggle) базовой модели (линейной регрессии) составила 51.5%

## 2. RandomForestRegressor

**По совету эксперта (и по мнению участников команды) нужно подобрать коэффициент, учитывающий изменения динамики ценообразования на рынке в среднем. Предположим, что к настоящему моменту цены в среднем выросли на 15%, то есть все предсказания мы будем умножать на 0.85.**

In [9]:
reg_tree = RandomForestRegressor()
reg_tree.fit(X_train, np.log(y_train))

y_pred = (np.exp(reg_tree.predict(X_test))//1000)*1000

# ошибка на тренировочной выборке 0.93%
mean_absolute_percentage_error(y_test, y_pred)

0.9319543963456719

In [ ]:
# делаем сабмит
res_sub = (np.exp(reg_tree.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub*0.85)//1000)*1000

#sample_submission.to_csv('reg_tree_15per.csv', index = False)

Submission: также применим снижение предсказанной цены на 15%, после этого **МАРЕ на Каггле для RandomForestRegressor - 32.8%**. 

## 3. ExtraTreesRegressor

In [80]:
reg_xtree = ExtraTreesRegressor()
reg_xtree.fit(X_train, np.log(y_train))

y_pred = (np.exp(reg_xtree.predict(X_test))//1000)*1000

# ошибка на тренировочной выборке 0.82%
mean_absolute_percentage_error(y_test, y_pred)

0.824799247320334

In [ ]:
# делаем сабмит
res_sub = (np.exp(reg_xtree.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub*0.85)//1000)*1000

#sample_submission.to_csv('reg_xtree_15per.csv', index = False)

Submission: также применим снижение предсказанной цены на 15%, после этого **МАРЕ на Каггле для ExtraTreesRegressor - 35.07%**. Самый слабый результат. Подбирать оптимальные параметры не будем, чтобы сэкономить время и вычислительную помщность наших ПК :)

## 4. GradientBoostingRegressor

In [10]:
reg_gbr = GradientBoostingRegressor()
reg_gbr.fit(X_train, np.log(y_train))

y_pred = (np.exp(reg_gbr.predict(X_test))//1000)*1000

# ошибка на тренировочной выборке 5.82%
mean_absolute_percentage_error(y_test, y_pred)

5.827961761750432

In [85]:
# делаем сабмит
res_sub = (np.exp(reg_gbr.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub*0.85)//1000)*1000

#sample_submission.to_csv('reg_gbr_15per.csv', index = False)

Submission: также применим снижение предсказанной цены на 15%, после этого **МАРЕ на Каггле для GradientBoostingRegressor - 32.7%**.

## 5. CatboostRegressor

In [90]:
reg_cat = CatBoostRegressor()
reg_cat.fit(X_train, np.log(y_train), verbose=False)

y_pred = (np.exp(reg_cat.predict(X_test))//1000)*1000

# ошибка на тренировочной выборке 1.13%
mean_absolute_percentage_error(y_test, y_pred)

1.1355335880832855

In [92]:
# делаем сабмит
res_sub = (np.exp(reg_cat.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub*0.85)//1000)*1000

#sample_submission.to_csv('reg_cat_15per.csv', index = False)

Submission: также применим снижение предсказанной цены на 15%, после этого **МАРЕ на Каггле для CatBoost - 26.2%**. 

**Лучший результат среди моделей с дефолтными параметрами!**

## 6. XGBRegressor

In [101]:
reg_xgb = xgb.XGBRegressor()
reg_xgb.fit(X_train, np.log(y_train))

y_pred = (np.exp(reg_xgb.predict(X_test))//1000)*1000

# ошибка на тренировочной выборке 1.17%
mean_absolute_percentage_error(y_test, y_pred)

1.174612308583004

In [102]:
# делаем сабмит
res_sub_xgb = (np.exp(reg_xgb.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub_xgb*0.85)//1000)*1000

#sample_submission.to_csv('reg_xgb_15per.csv', index = False)

Submission: также применим снижение предсказанной цены на 15%, после этого **МАРЕ на Каггле для XGBRegressor - 27.3%**. Второй алгоритм по качеству предсказания на тестовой выборке.

### Промежуточные выводы:
* Применение более сложных алгоритмов существенно улучшило метрику качества предсказания, особенно на тренировочной выброке.
* Несмотря на то, что модели прекрасно справляются с задачей предсказания цен на тренировочной выборке, тестовый результат на платформе оказывается заметно хуже, что может быть вызвано большим временным интервалом между парсингом баз.
* Для того, чтобы учесть изменившеюся динамику ценообразования, мы приняли решение домножать предсказанные цены на определенный коэффициент, то есть намеренно понижать предсказанные цены, что повысило точность модели на тестовых данных платформы. 
* Наконец, использование более сложных моделей даже с дефолтными параметрами позволило в лучшем случае снизить MAPE на тестовой выборке примерно в 2 раза по сравнению с базовой моделью линейной регрессии.

## Подбор гиперпараметров:

### GradientBoostingRegressor

In [21]:
# random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 8)],
# 'max_features': ['auto', 'sqrt', 'log2'],
# 'max_depth': [int(x) for x in np.linspace(5, 15, num = 6)] + [None],
# 'min_samples_split': [2, 5, 10],
# 'min_samples_leaf': [1, 2, 4]}

#gbr = GradientBoostingRegressor()
# gbr_optimal = RandomizedSearchCV(estimator = gbr, param_distributions = random_grid, n_iter = 10, cv = 3,\
# verbose=10, random_state=42, n_jobs = -1)
#gbr_optimal.fit(X_train, np.log(y_train))
# gbr_optimal.best_params_

In [ ]:
# optimal parameters

# {'n_estimators': 271,
# 'min_samples_split': 10,
# 'min_samples_leaf': 4,
# 'max_features': 'log2',
# 'max_depth': 11}

In [11]:
best_gbr = GradientBoostingRegressor(random_state=42, n_estimators=271,
                                     min_samples_split=10, min_samples_leaf=4, max_features='log2', max_depth=11)

best_gbr.fit(X_train, np.log(y_train))

predict_gbr = (np.exp(best_gbr.predict(X_test))//1000)*1000
mean_absolute_percentage_error(y_test, predict_gbr)

0.9850763292248863

Подбор гиперпараметров заметно улучшил результат для GradientBoostingRegressor: было MAPE 5.82%, а стало 0.98%. Результат на Kaggle:

In [12]:
# делаем сабмит
res_sub = (np.exp(best_gbr.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub*0.85)//1000)*1000

#sample_submission.to_csv('best_gbr_15per.csv', index = False)

Submission: **МАРЕ на Каггле для GradientBoostingRegressor было 32.7%, а стало - 27.4%, что говорит о заметном улучшении.**

### CatBoostRegressor

In [11]:
#model = CatBoostRegressor(iterations=50, loss_function='MAPE', metric_period=10)

# grid = {'learning_rate': [0.13, 0.14, 0.15]
# ,'depth': [10,11,12]
# ,'l2_leaf_reg': [7, 7.5, 8]
# ,'random_strength': [0.2, 0.3, 0.4]}

#grid_search_result = model.grid_search(grid,X=X_train,y=np.log(y_train),plot=False,verbose=False,cv=3)

# grid_search_result['params']

# optimal parameters
#{'depth': 11, 'l2_leaf_reg': 8, 'random_strength': 0.3, 'learning_rate': 0.15}

In [16]:
cb_optimal = CatBoostRegressor(iterations=100,
                               random_seed=42,
                               eval_metric='MAPE',
                               custom_metric=['R2', 'MAE'],
                               silent=True,
                               learning_rate=0.15, depth=11,
                               l2_leaf_reg=8, random_strength=0.3)

cb_optimal.fit(X_train, np.log(y_train),
               eval_set=(X_test, np.log(y_test)),
               verbose=False,
               use_best_model=True,
               plot=False)

predict_cb = (np.exp(cb_optimal.predict(X_test))//1000)*1000
mean_absolute_percentage_error(y_test, predict_cb)

1.589290554304865

На тренировочной выборке метрика ухудшилась: была 1.13% с дефолтными параметрами, а стала 1.58%. Проверим на Кагле:

In [18]:
# делаем сабмит
res_sub = (np.exp(cb_optimal.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub*0.85)//1000)*1000

#sample_submission.to_csv('best_cb_15per.csv', index = False)

На Кагле **метрика ухудшилась: стала 32.9%, а была 26.2% с дефолтными гиперпараметрами, что говорит об ухудшении результата.**

### RandomForestRegressor

In [12]:
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
#max_features = ['auto', 'sqrt']
#max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
#min_samples_split = [2, 5, 10]
#min_samples_leaf = [1, 2, 4]
#bootstrap = [True, False]
# random_grid = {'n_estimators': n_estimators,
# 'max_features': max_features,
# 'max_depth': max_depth,
# 'min_samples_split': min_samples_split,
# 'min_samples_leaf': min_samples_leaf,
# 'bootstrap': bootstrap}

#rf = RandomForestRegressor(random_state=42)
# rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100,
# cv=5, verbose=2, random_state=42, n_jobs=-1)

# best parameters

# {'n_estimators': 400,
# 'min_samples_split': 2,
# 'min_samples_leaf': 1,
# 'max_features': 'sqrt',
# 'max_depth': None,
# 'bootstrap': False}

In [13]:
best_rf = RandomForestRegressor(random_state=42, n_estimators=400, min_samples_split=2,
                                min_samples_leaf=1, max_features='sqrt', max_depth=None, bootstrap=False)

best_rf.fit(X_train, np.log(y_train))

predict_rf = (np.exp(best_rf.predict(X_test))//1000)*1000
mean_absolute_percentage_error(y_test, predict_rf)

0.8872284820731196

На тренировочной выборке метрика немного улучшилась: была 0.93% с дефолтными параметрами, а стала 0.88%. Проверим на Кагле:

In [ ]:
# делаем сабмит
res_sub = (np.exp(best_rf.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub*0.85)//1000)*1000

#sample_submission.to_csv('best_rf_15per.csv', index = False)

На Кагле **метрика немного улучшилась: стала 31.05%, а была 32.8% с дефолтными гиперпараметрами**, тем не менее RF по прежнему среди наиболее слабых моделей для данной задачи.

### XGBRegressor

In [14]:
# оптимальные параметры XGB заимствованы у других участников соревнования
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                           learning_rate=0.05, max_depth=12, alpha=1, n_estimators=1000)
xgb_reg.fit(X_train, np.log(y_train))

y_pred = (np.exp(xgb_reg.predict(X_test))//1000)*1000

mean_absolute_percentage_error(y_test, y_pred)

0.8789908458512913

На тренировочной выборке метрика немного улучшилась: была 1.17% с дефолтными параметрами, а стала 0.87%. Проверим на Кагле:

In [ ]:
# делаем сабмит
res_sub = (np.exp(xgb_reg.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub*0.85)//1000)*1000

#sample_submission.to_csv('best_xgb_15per.csv', index = False)

На Кагле **метрика немного улучшилась: стала 25.7%, а была 27.3% с дефолтными гиперпараметрами.** На данном этапе модель XGBRegressor c оптимальными гиперпараметрами показывает наилучший результат.

### Промежуточные рузельтаты:
* Не все модели улучшились после подбора оптимальных гиперпараметров.
* Для XGBRegressor подбор гиперпараметров оказался наиболее продуктивным: модель показывает наилучший результат на платформе.

## Bagging & Stacking

Работаем с нашими лучшими моделями на предыдущих шагах - **CatBoost (дефолтные параметры)** и **XGBRegressor (оптимальные параметры)**.

### Bagging для дефолтного CatBoost

In [16]:
from sklearn.datasets import make_regression

#reg_cat = CatBoostRegressor(verbose=False)
#regr = BaggingRegressor(base_estimator=reg_cat,n_estimators=10, random_state=42).fit(X_train, np.log(y_train))

#y_pred = (np.exp(regr.predict(X_test))//1000)*1000
# mean_absolute_percentage_error(y_test, y_pred)   #1.11

1.1110003273783757

Метрика на базовом CatBoost была 1.13%, с бэггингом стала - 1.11%. На Кагле:

In [ ]:
# делаем сабмит
res_sub = (np.exp(regr.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub*0.85)//1000)*1000

#sample_submission.to_csv('sub_cat_bag.csv', index = False)

На Кагле **метрика немного улучшилась: стала 25.9%, а была на дефолтном CatBoost 26.2%**

### Bagging для оптимального XGBRegressor

In [18]:
#regr_bag_xgb = BaggingRegressor(base_estimator=xgb_reg,n_estimators=10, random_state=42).fit(X_train, np.log(y_train))
#y_pred = (np.exp(regr_bag_xgb.predict(X_test))//1000)*1000

# mean_absolute_percentage_error(y_test, y_pred) #0.88

Метрика на оптимальном XGBRegressor была 0.87%, с бэггингом стала - 0.88%. На Кагле:

In [ ]:
# делаем сабмит
res_sub = (np.exp(regr_bag_xgb.predict(test_Kaggle))//1000)*1000
sample_submission['price'] = ((res_sub*0.85)//1000)*1000

#sample_submission.to_csv('sub_xgb_bag.csv', index = False)

На Кагле **метрика немного улучшилась: с бэггинг 24.8%, а была на просто оптимальном XGB 25.7%**. На данном этапе это наш лучший результат!

Наконец, мы решили вручную перебрать коэффициент, на который мы домножаем цены (уменьшаем цены) для нашей лучшей модели и проверить ошибку на платформе. Получили следующие результаты:

* Без коррекции - МАРЕ 28.09%
* Коррекция на 10% - МАРЕ 24.9%
* **Коррекция на 13% - МАРЕ 24.76%** - наш лучший результат в целом.
* Коррекция на 15% - МАРЕ 24.88%
* Коррекция на 17% - МАРЕ 25.15%
* Коррекция на 22% - МАРЕ 26.51%


### Stacking

In [19]:
# Вариант 1

# estimators=[('regr', BaggingRegressor(base_estimator=CatBoostRegressor(),n_estimators=10, random_state=42)),\
# ('regr_bag_xgb', BaggingRegressor(base_estimator=xgb_reg,n_estimators=10, random_state=42))]

#st_ensemble = StackingRegressor(estimators=estimators,final_estimator = CatBoostRegressor())

#st_ensemble.fit(X_train, np.log(y_train))

#y_pred = (np.exp(st_ensemble.predict(X_test))//1000)*1000

# mean_absolute_percentage_error(y_test, y_pred)  #1.50

Сабмит на платформу дал ошибку 25.21%, то есть просто бэггинг на XGB был лучше.

In [ ]:
# Вариант 2

# estimators = [('regr', BaggingRegressor(base_estimator=CatBoostRegressor(), n_estimators=10, random_state=42)),
# ('regr_bag_xgb', BaggingRegressor(base_estimator=xgb_reg, n_estimators=10, random_state=42))]

# st_ensemble = StackingRegressor(estimators=estimators, final_estimator=GradientBoostingRegressor(
# random_state=42, n_estimators=271, min_samples_split=10, min_samples_leaf=4, max_features='log2', max_depth=11))

#st_ensemble.fit(X_train, np.log(y_train))

#y_pred = (np.exp(st_ensemble.predict(X_test))//1000)*1000

# mean_absolute_percentage_error(y_test, y_pred) #1.10

МАРЕ на платформе получилось 26.22% - то есть опять хуже чем XGB с бэггинге.

In [ ]:
# Вариант 3

# stimators = [('b_gbr', BaggingRegressor(GradientBoostingRegressor(random_state=42, n_estimators=800, min_samples_split=5,
# min_samples_leaf=4, max_features='sqrt', max_depth=9),
# n_estimators=3, n_jobs=1, random_state=42)),
# ('xgb', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
# learning_rate=0.05, max_depth=12, alpha=1, n_estimators=1000))]

# st_ensemble_kag = StackingRegressor(estimators=estimators,\
# final_estimator=CatBoostRegressor(iterations=5000, random_seed=42, eval_metric='MAPE',custom_metric=[
# 'R2', 'MAE'],silent=True, learning_rate=0.13, depth=12, l2_leaf_reg=8, random_strength=0.3))

#st_ensemble_kag.fit(X_train, np.log(y_train))

#y_pred = (np.exp(st_ensemble_kag.predict(X_test))//1000)*1000

# mean_absolute_percentage_error(y_test, y_pred)  #1.51

МАРЕ на Каггле 27.33% - ещё хуже.

# Общие выводы

**Тестовая база данных:**

* База довольно "грязная", в ней много дублирующих друг друга признаков, но самое главное - нет понимания того КАК и самое главное КОГДА она была сделана. Цены на рынке подержанных машин в течение последнего года сильно изменялись и нам пришлось вводить корректирующий коэффициент (оптимальный - 13%) на предсказания, сделанные по актуальной базе. 
* Если бы мы знали когда была изготовлена база, то мы могли бы попытаться сделать корректирующие коэффициенты отдельно по каждой марке машин или по группам марок, что существенно улучшило бы показатели метрики. Кроме того понимание кода, который использовался при парсинге тестовой базы, помогло бы актуальную базу данных спарсить в виде более похожем на тестовую базу.

**Тренировочная база данных:**

* Парсинг того объема данных, который использовался в работе, занял приблизительно 30 часов, что очень времязатратно и требует больших вычислительных мощностей и безперебойной работы интернета, что не всегда легко доступно.
* У нас не хватило времени поработать с категорией "комплектация". Наличие некоторых опций в комплектации существенно влияет на цену и на ее изменение со временм (к примеру, кожаный салон будет дороже тканевого и наличие этой опции при прочих равных сделает машину более дорогой. Но времени пройтись по представительствам разных марок и спросить какие из около 300 представленных в базе опций будут в итоге существенно вляить на цену, а какие нет - нам не удалось.

**Метрики и результаты:**

* Мы провели многочисленные проверки работы моделей как с дефолтными настройками, так и с подобранными гиперпараметрами, использовали бэггинг и стеккинг. Последние два метода использовали не для всех моделей и их сочетаний в силу высокой ресурсоемкости процессов. 
* Наилучшие результаты показал CatboostRegressor на дефолтных настройках и XGBRegressor с применением бэггинга.
* Серьезная разница между очень низкой метрикой МАРЕ на тренинговых данных и высокой на тестовых может объясняться несколькими причинами - переобучением модели на тренинге (но выявить это переобучение нам не удалось) и серьезным изменением рынка подержанных машин за прошедший год (динамика ценообразования).
* Финальный результат метрики MAPE - 24.76% (на тестовых данных Kaggle) - и это наша лучшная модель: Бэггинг для оптимального XGBRegressor c 13%-ой поправкой цены.